In [76]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import gensim

import torch
from torch import nn
import gensim
import fasttext
from gensim.models import FastText
import random
import operator
from nltk.tokenize import word_tokenize
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import pickle
import joblib
import nltk

In [9]:
df_discharge_train = pd.read_csv("../clinicalBERT/data/discharge/train.csv")
df_discharge_val = pd.read_csv("../clinicalBERT/data/discharge/val.csv")
df_discharge_test = pd.read_csv("../clinicalBERT/data/discharge/test.csv")
df_3day_train = pd.read_csv("../clinicalBERT/data/3days/train.csv")
df_3day_val = pd.read_csv("../clinicalBERT/data/3days/val.csv")
df_3day_test = pd.read_csv("../clinicalBERT/data/3days/test.csv")

In [10]:
# concat em all

dfs = [df_discharge_train,df_discharge_val, df_discharge_test, df_3day_train, df_3day_val, df_3day_test]
df_all = pd.concat(dfs)
df_all.shape

(91353, 4)

In [19]:
training_data = df_all.TEXT.values

In [18]:
training_data.shape

(91353,)

In [78]:
# training_corpus = df_all.TEXT.apply(lambda x: x.split(" "))
# training_corpus

In [79]:
tokenizer = nltk.RegexpTokenizer(r"\w+").tokenize
training_corupus = df_all.TEXT.apply(tokenizer)
model = gensim.models.Word2Vec(sentences=training_corpus, vector_size=200, window=5, min_count=1, workers=4)

In [80]:
model.save("./data/mimic3/new_mimic_word2vec_200.model")

In [81]:
model = gensim.models.KeyedVectors.load("./data/mimic3/new_mimic_word2vec_200.model")

In [82]:
weights = torch.FloatTensor(model.wv.vectors)
print("weights vector shape: ", weights.shape)

weights vector shape:  torch.Size([337754, 200])


In [41]:
model.wv.key_to_index

{'and': 0,
 'the': 1,
 'to': 2,
 'of': 3,
 'with': 4,
 'for': 5,
 'in': 6,
 'is': 7,
 'on': 8,
 'was': 9,
 'no': 10,
 '-': 11,
 'a': 12,
 'at': 13,
 '.': 14,
 'pt': 15,
 'as': 16,
 'left': 17,
 'or': 18,
 'patient': 19,
 'mg': 20,
 'he': 21,
 'this': 22,
 'ml': 23,
 'right': 24,
 'po': 25,
 'am': 26,
 'there': 27,
 'not': 28,
 'are': 29,
 'blood': 30,
 'she': 31,
 'from': 32,
 '/': 33,
 '#': 34,
 'tablet': 35,
 'be': 36,
 'his': 37,
 'has': 38,
 'pm': 39,
 'one': 40,
 'chest': 41,
 'sig:': 42,
 'but': 43,
 ',': 44,
 'her': 45,
 'by': 46,
 'had': 47,
 '2': 48,
 'ct': 49,
 'were': 50,
 'given': 51,
 'normal': 52,
 '5': 53,
 'daily': 54,
 's/p': 55,
 '1': 56,
 'pain': 57,
 '(1)': 58,
 'other': 59,
 'that': 60,
 'iv': 61,
 'medical': 62,
 'an': 63,
 'have': 64,
 'history:': 65,
 'which': 66,
 'will': 67,
 'acute': 68,
 'discharge': 69,
 'continue': 70,
 'after': 71,
 'also': 72,
 'history': 73,
 '4': 74,
 'no(t)': 75,
 'aortic': 76,
 'per': 77,
 'respiratory': 78,
 'you': 79,
 'prior': 80,

In [83]:
model.wv.key_to_index[',']

44

In [84]:
sims = model.wv.most_similar('lung', topn=10)  # get other similar words
sims

[('lung,', 0.603064239025116),
 ('airspace', 0.5790968537330627),
 ('lungs,', 0.5700814723968506),
 ('lungs', 0.5676866173744202),
 ('hemithorax', 0.5621927976608276),
 ('lung.', 0.5619516968727112),
 ('lobe', 0.5579512715339661),
 ('mid-lung', 0.5488782525062561),
 ('-lung', 0.50739586353302),
 ('perihilar', 0.505617618560791)]

In [66]:
model.wv.most_similar('heart', topn=10)

[('thrive,', 0.5690180659294128),
 ('bradicardia;renal', 0.5227065682411194),
 ('as/chf/renal', 0.519071102142334),
 ('cardiac', 0.5075947046279907),
 ('hepatopathy.', 0.5029286742210388),
 ('syncope-renal', 0.5008339881896973),
 ('multiorgan', 0.49661725759506226),
 ('#1:respiratory', 0.4908979833126068),
 ('hepatopathy', 0.47541025280952454),
 ('atrioventricular', 0.47126269340515137)]

In [67]:
model.wv.most_similar('leg', topn=10)

[('arm', 0.7820788025856018),
 ('hand', 0.7268771529197693),
 ('foot', 0.7242743968963623),
 ('leg.', 0.704299807548523),
 ('arm,', 0.6844622492790222),
 ('leg,', 0.683937668800354),
 ('ankle', 0.6665143370628357),
 ('thigh', 0.6509072184562683),
 ('rle', 0.6457663178443909),
 ('forearm', 0.6448269486427307)]